In [21]:
import numpy
#import tensorflow as tf
from tensorflow import keras
import pandas as pd

for tpu

In [1]:
%tensorflow_version 2.x
import tensorflow as tf
print("Tensorflow version " + tf.__version__)

try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
  print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
  raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')

tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

Tensorflow version 2.8.2
Running on TPU  ['10.81.58.106:8470']
INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Initializing the TPU system: grpc://10.81.58.106:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.81.58.106:8470


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


for gpu (dont run unless on gpu)

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

build

inception pre-tranined

In [2]:
  include_top=True
  weights="imagenet"
  input_shape=(299, 299, 3)
  pooling=True
  classes=1000
  classifier_activation="softmax"

In [10]:
with tpu_strategy.scope():
  inception_m = tf.keras.applications.InceptionV3(
      include_top=True,
      weights="imagenet",
      input_tensor=None,
      input_shape=input_shape,
      pooling=True,
      classes=1000,
      classifier_activation="softmax",
  )

In [4]:
inception_m.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 299, 299, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 149, 149, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 149, 149, 32  96         ['conv2d[0][0]']                 
 alization)                     )                                                      

In [11]:
with tpu_strategy.scope():
  x = tf.keras.layers.Dense(512,activation="relu",use_bias=True,)(inception_m.layers[-2].output)
  x = tf.keras.layers.Dense(64,activation="relu",use_bias=True,)(x)
  y = tf.keras.layers.Dense(5,activation="softmax",use_bias=True)(x)

  tt = tf.keras.models.Model(inception_m.input, y) #ensure the input is the layer of the source model

In [13]:
tt.compile(optimizer='adam', loss='categorical_crossentropy')

In [14]:
tt.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 299, 299, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_94 (Conv2D)             (None, 149, 149, 32  864         ['input_2[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization_94 (BatchN  (None, 149, 149, 32  96         ['conv2d_94[0][0]']              
 ormalization)                  )                                                           

In [10]:
for layer in range(len(tt.layers) -5):
  tt.layers[layer].trainable = False

In [15]:
tt.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 299, 299, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_94 (Conv2D)             (None, 149, 149, 32  864         ['input_2[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization_94 (BatchN  (None, 149, 149, 32  96         ['conv2d_94[0][0]']              
 ormalization)                  )                                                           

cnn model

In [16]:
from tensorflow.keras import Sequential

In [17]:
 with tpu_strategy.scope():
  model = Sequential([tf.keras.layers.Conv2D(
      filters = 32,
      kernel_size= 5,
      strides=(1, 1),
      padding='valid',
      data_format=None,
      dilation_rate=(1, 1),
      groups=1,
      activation="relu",
      use_bias=True,
      kernel_initializer='glorot_uniform',
      bias_initializer='zeros',
      kernel_regularizer=None,
      bias_regularizer=None,
      activity_regularizer=None,
      kernel_constraint=None,
      bias_constraint=None,
  ), tf.keras.layers.MaxPool2D(
      pool_size=(3, 3),
      strides=None,
      padding='valid',
      data_format=None,
  ),
  tf.keras.layers.BatchNormalization(
      axis=-1,
      momentum=0.99,
      epsilon=0.001,
      center=True,
      scale=True,
      beta_initializer='zeros',
      gamma_initializer='ones',
      moving_mean_initializer='zeros',
      moving_variance_initializer='ones',
      beta_regularizer=None,
      gamma_regularizer=None,
      beta_constraint=None,
      gamma_constraint=None,
  ),
  tf.keras.layers.Conv2D(
      filters = 64,
      kernel_size = 3,
      strides=(1, 1),
      padding='valid',
      data_format=None,
      dilation_rate=(1, 1),
      groups=1,
      activation="relu",
      use_bias=True,
      kernel_initializer='glorot_uniform',
      bias_initializer='zeros',
      kernel_regularizer=None,
      bias_regularizer=None,
      activity_regularizer=None,
      kernel_constraint=None,
      bias_constraint=None,
  ),
  tf.keras.layers.Conv2D(
      filters = 128,
      kernel_size = 3,
      strides=(1, 1),
      padding='valid',
      data_format=None,
      dilation_rate=(1, 1),
      groups=1,
      activation="relu",
      use_bias=True,
      kernel_initializer='glorot_uniform',
      bias_initializer='zeros',
      kernel_regularizer=None,
      bias_regularizer=None,
      activity_regularizer=None,
      kernel_constraint=None,
      bias_constraint=None,
  ),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(
      512,
      activation="relu",
      use_bias=True,
      kernel_initializer='glorot_uniform',
      bias_initializer='zeros',
      kernel_regularizer=None,
      bias_regularizer=None,
      activity_regularizer=None,
      kernel_constraint=None,
      bias_constraint=None,
  ),
  tf.keras.layers.Dense(
      1024,
      activation="relu",
      use_bias=True,
      kernel_initializer='glorot_uniform',
      bias_initializer='zeros',
      kernel_regularizer=None,
      bias_regularizer=None,
      activity_regularizer=None,
      kernel_constraint=None,
      bias_constraint=None,
  ),
  tf.keras.layers.Dense(
      64,
      activation="relu",
      use_bias=True,
      kernel_initializer='glorot_uniform',
      bias_initializer='zeros',
      kernel_regularizer=None,
      bias_regularizer=None,
      activity_regularizer=None,
      kernel_constraint=None,
      bias_constraint=None,
  ),
  tf.keras.layers.Dense(
      5,
      activation="softmax",
      use_bias=True,
      kernel_initializer='glorot_uniform',
      bias_initializer='zeros',
      kernel_regularizer=None,
      bias_regularizer=None,
      activity_regularizer=None,
      kernel_constraint=None,
      bias_constraint=None,
  )])

In [18]:
model.compile(optimizer='adam', loss='categorical_crossentropy')

data preprosesssing

In [19]:
data = tf.keras.preprocessing.image.ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    zca_whitening=False,
    zca_epsilon=1e-06,
    rotation_range=0.3,
    width_shift_range=0.3,
    height_shift_range=0.33,
    brightness_range=None,
    shear_range=0.1,
    zoom_range=0.2,
    channel_shift_range=0.3,
    fill_mode='nearest',
    cval=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    rescale=1./255,
    preprocessing_function=None,
    validation_split=0.2,
)

In [22]:
traindf = pd.read_csv('/content/drive/MyDrive/Cassava Leaf disease dataset/Data set 1/train.csv',dtype=str)

In [23]:
traindf

,image_id,label
0,1000015157.jpg,0
1,1000201771.jpg,3
2,100042118.jpg,1
3,1000723321.jpg,1
4,1000812911.jpg,3
...,...,...
21392,999068805.jpg,3
21393,999329392.jpg,3
21394,999474432.jpg,1
21395,999616605.jpg,4


In [ ]:
train_generator = data.flow_from_dataframe(
    dataframe=traindf,
    directory="/content/drive/MyDrive/Cassava Leaf disease dataset/Data set 1/train_images",
    x_col="image_id",
    y_col="label",
    subset='training', # important for spliting training set as all datais from same data set
    target_size=(299, 299),
    color_mode="rgb",
    batch_size=32,
    class_mode="categorical",
    shuffle=True,
    seed=42
)

valid_generator = data.flow_from_dataframe(
    dataframe=traindf,
    directory="/content/drive/MyDrive/Cassava Leaf disease dataset/Data set 1/train_images",
    x_col="image_id",
    y_col="label",
    subset='validation', # important for spliting validation set as all datais from same data set
    target_size=(299, 299),
    color_mode="rgb",
    batch_size=32,
    class_mode="categorical",
    shuffle=True,
    seed=42
)

In [ ]:
model.fit(
    x=train_generator,
    batch_size=32,
    epochs=3,
    verbose='auto',
    callbacks=None,
    validation_data=valid_generator,
    shuffle=True,
    class_weight=None,
    sample_weight=None,
    initial_epoch=0,
    steps_per_epoch=None,
    validation_steps=None,
    validation_batch_size=None,
    validation_freq=1,
    max_queue_size=10,
    workers=1,
    use_multiprocessing=False
)

In [ ]:
tt